### Imports

In [1]:
# main library
import gspread

# authentication library that google drive API uses. This assumes that you have already set up a service account for google API
from oauth2client.service_account import ServiceAccountCredentials

# generic other imports
import pandas as pd
import numpy as np
import itertools
import math

### Authenticating and logging in

In [2]:
# use credentials to create a client that interacts with the Google Drive API

scope = [
    'https://spreadsheets.google.com/feeds',
    'https://www.googleapis.com/auth/drive'
]
headers = gspread.httpsession.HTTPSession(headers={'Connection':'Keep-Alive'})


# client secret json is the key I got from Google API setup page
# Can read more here https://www.twilio.com/blog/2017/02/an-easy-way-to-read-and-write-to-a-google-spreadsheet-in-python.html
creds = ServiceAccountCredentials.from_json_keyfile_name('client_secret.json', scope)
client = gspread.Client(auth = creds, http_session=headers)
client.login()


# Find a workbook by name and open the first sheet (sheet1 means the first sheet)
# Make sure you use the right name here.

# grab the first sheet
sheet = client.open("Copy of Legislators 2017").sheet1

# grab other sheets as applicable

### Read in the file that needs to be written

### Construct ranges to batch update file

In [3]:
df = pd.read_csv("Active_List.csv",header=None,encoding='ISO-8859-1').fillna(value="")

l = list(itertools.chain(*(df.values.tolist())))

# batch update n rows at once
n = 500

rows = df.shape[0]
cols = df.shape[1]
split = math.ceil(df.shape[0]/n)

# split list in ranges
range_list = []
diff = n*cols

for it in range(0,split):
    if (it != (split-1)):
        range_start = it*diff
        range_end = range_start + diff
    else:
        range_start = it*diff
        range_end = len(l)
    
    ran = [range_start,range_end]
    range_list.append(ran)

In [4]:
range_list

[[0, 26000], [26000, 52000], [52000, 78000], [78000, 95368]]

### Define function to get the final range of cells

In [5]:
def get_range(row,col):
    from_range = 'A1'
    if col <=26:
        to = chr(col + ord('A')-1)
    else:
        diff = col - 26
        to = 'A' + chr(ord('A')+diff-1)
    to_range = to + str(row)
    return (from_range + ':' + to_range)

### Final piece of code

In [6]:
range_test = get_range(rows,cols)
cell_list_test = sheet.range(range_test)

print("Updating cells..")

for i in range(0,len(l)): 
    cell_list_test[i].value = l[i]
    i += 1

print ("Cells updated, starting batch update to google docs..")

for iteration in range(0,len(range_list)):
    print ("Updating rows",int(range_list[iteration][0]/cols),"to",int(range_list[iteration][1]/cols))
    sheet.update_cells(cell_list_test[range_list[iteration][0]:range_list[iteration][1]])

print("Success!")

Updating cells..
Cells updated, starting batch update to google docs..
Updating rows 0 to 500
Updating rows 500 to 1000
Updating rows 1000 to 1500
Updating rows 1500 to 1834
Success!


### Other functionality

In [ ]:
# update a cell on a sheet
sheet.update_cell(1,1,"Writing to spreadsheet")

#creating a new spreadsheet
sh = client.create('New Spreadsheet')

#giving myself permission to view/edit the newly created sheet
client.insert_permission(
    sh.id,
    'csathri@purestorage.com',
    perm_type='user',
    role='writer'
)

# selecting the first sheet
sh1 = sh.sheet1

#updating the first cell
sh1.update_cell(1,1,"works?")